# **第2章 Python数理最適化チュートリアル**

## **2.1 連立一次方程式をPythonの数理最適化ライブラリで解く**

### 全体のコード

In [1]:
import pulp

problem = pulp.LpProblem('SLE', pulp.LpMaximize)

x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

problem += 120 * x + 150 * y == 1440
problem += x + y == 10

status = problem.solve()

print('Status:', pulp.LpStatus[status])
print('x=', x.value(), 'y=', y.value())

Using license file /Library/gurobi903/gurobi.lic
No parameters matching '_test' found
Status: Optimal
x= 2.0 y= 8.0


### 本書における逐次実行

In [2]:
# PythonライブラリPuLPの取り込み
import pulp

In [3]:
# 数理モデルの定義
problem = pulp.LpProblem('SLE', pulp.LpMaximize)
problem # 追記

SLE:
MAXIMIZE
None
VARIABLES

In [4]:
# 変数の定義
x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

In [5]:
# 制約式の定義
problem += 120 * x + 150 * y == 1440
problem += x + y == 10
problem # 追記

SLE:
MAXIMIZE
None
SUBJECT TO
_C1: 120 x + 150 y = 1440

_C2: x + y = 10

VARIABLES
x free Continuous
y free Continuous

In [6]:
# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

Status: Optimal


In [7]:
# 最適化結果の表示
print('x=', x.value(), 'y=', y.value())

x= 2.0 y= 8.0


## **2.2 線形計画問題をPythonの数理最適化ライブラリで解く**

### 全体のコード

In [8]:
import pulp

problem = pulp.LpProblem('LP', pulp.LpMaximize)

x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

problem += 1 * x + 3 * y <= 30
problem += 2 * x + 1 * y <= 40
problem += x >= 0
problem += y >= 0
problem.objective = x + 2 * y

status = problem.solve()

print('Status:', pulp.LpStatus[status])
print('x=', x.value(), 'y=', y.value(), 'obj=', problem.objective.value())

Status: Optimal
x= 18.0 y= 4.0 obj= 26.0


### 本書における逐次実行

In [9]:
# PythonライブラリPuLPの取り込み
import pulp

In [10]:
# 数理最適化モデルの定義
problem = pulp.LpProblem('LP', pulp.LpMaximize)
problem

LP:
MAXIMIZE
None
VARIABLES

In [11]:
# 変数の定義
x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

In [12]:
# 制約式の定義
problem += 1 * x + 3 * y <= 30
problem += 2 * x + 1 * y <= 40
problem += x >= 0
problem += y >= 0

In [13]:
# 目的関数の定義
problem.objective = x + 2 * y
problem # 追記

LP:
MAXIMIZE
1*x + 2*y + 0
SUBJECT TO
_C1: x + 3 y <= 30

_C2: 2 x + y <= 40

_C3: x >= 0

_C4: y >= 0

VARIABLES
x free Continuous
y free Continuous

In [14]:
# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

Status: Optimal


In [15]:
# 最適化結果の表示
print('x=', x.value(), 'y=', y.value(), 'obj=', problem.objective.value())

x= 18.0 y= 4.0 obj= 26.0


## **2.3 規模の大きな数理最適化問題をPythonの数理最適化ライブラリで解く**

### **線形計画問題**

### ①データのインポート

In [16]:
# データ処理のためのライブラリpandasとPythonライブラリPuLPの取り込み
import pandas as pd
import pulp

In [17]:
# stocks.csvからのデータ取得
stock_df = pd.read_csv('stocks.csv')
stock_df

,m,stock
0,m1,35
1,m2,22
2,m3,27


In [18]:
# requires.csvからのデータ取得
require_df = pd.read_csv('requires.csv')
require_df

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


In [19]:
# gains.csvからのデータ取得
gain_df = pd.read_csv('gains.csv')
gain_df

,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


### ②リストの定義

In [20]:
# 製品のリストの定義
P = gain_df['p'].tolist()
P

['p1', 'p2', 'p3', 'p4']

In [21]:
# 原料のリストの定義
M = stock_df['m'].tolist()
M

['m1', 'm2', 'm3']

### ③定数の定義

In [22]:
# 定数の定義:stock
stock = {row.m:row.stock for row in stock_df.itertuples()}

#stock = dict(zip(stock_df['m'], stock_df['stock']))
#stock = dict((row.m, row.stock) for row in stock_df.itertuples())
#stock = {row['m']:row['stock'] for i, row in stock_df.iterrows()} # 追記:iterrowsは低速なので避ける
#stock = stock_df.set_index('m').to_dict()['stock']
#stock = stock_df.set_index('m')['stock'].to_dict() # 追記
stock

{'m1': 35, 'm2': 22, 'm3': 27}

In [23]:
# 定数の定義:gain
gain = {row.p:row.gain for row in gain_df.itertuples()}
gain

{'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}

In [24]:
# 定数の定義:require
require = {(row.p,row.m):row.require for row in require_df.itertuples()}
require

{('p1', 'm1'): 2,
 ('p1', 'm2'): 0,
 ('p1', 'm3'): 1,
 ('p2', 'm1'): 3,
 ('p2', 'm2'): 2,
 ('p2', 'm3'): 0,
 ('p3', 'm1'): 0,
 ('p3', 'm2'): 2,
 ('p3', 'm3'): 2,
 ('p4', 'm1'): 2,
 ('p4', 'm2'): 2,
 ('p4', 'm3'): 2}

### ④線形計画問題の定義

In [25]:
# 数理最適化モデルの定義
problem = pulp.LpProblem('LP2', pulp.LpMaximize)

### ⑤変数の定義

In [26]:
# 変数の定義
x = pulp.LpVariable.dicts('x', P, cat='Continuous')

# 変数の逐次定義
#x = {}
#for p in P:
#    x[p] = pulp.LpVariable('x_{}'.format(p), cat='Continuous')

# f-strings(Python3.6以降)
#x = {}
#for p in P:
#    x[p] = pulp.LpVariable(f'x_{p}', cat='Continuous')    

# 辞書 & f-strings
# x = {p:pulp.LpVariable(f'x_{p}', cat='Continuous') for p in P}

### ⑥制約式の定義

In [27]:
# 生産量は0以上
for p in P:
    problem += x[p] >= 0

In [28]:
# 生産量は在庫の範囲
for m in M:
    problem += pulp.lpSum([require[p,m] * x[p] for p in P]) <= stock[m]

### ⑦目的関数の定義

In [29]:
# 目的関数の定義    
problem += pulp.lpSum([gain[p] * x[p] for p in P])
problem # 追記

LP2:
MAXIMIZE
3*x_p1 + 4*x_p2 + 4*x_p3 + 5*x_p4 + 0
SUBJECT TO
_C1: x_p1 >= 0

_C2: x_p2 >= 0

_C3: x_p3 >= 0

_C4: x_p4 >= 0

_C5: 2 x_p1 + 3 x_p2 + 2 x_p4 <= 35

_C6: 2 x_p2 + 2 x_p3 + 2 x_p4 <= 22

_C7: x_p1 + 2 x_p3 + 2 x_p4 <= 27

VARIABLES
x_p1 free Continuous
x_p2 free Continuous
x_p3 free Continuous
x_p4 free Continuous

### 実行

In [30]:
# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

Status: Optimal


In [31]:
# 計算結果の表示
for p in P:
    print(p, x[p].value())

print('obj=', problem.objective.value())

p1 12.142857
p2 3.5714286
p3 7.4285714
p4 0.0
obj= 80.42857099999999


### ⑧実装した数理最適化モデルのまとめ

In [32]:
import pandas as pd
import pulp

# データの取得
require_df = pd.read_csv('requires.csv')
stock_df = pd.read_csv('stocks.csv')
gain_df = pd.read_csv('gains.csv')

# 集合の定義
P = gain_df['p'].tolist()
M = stock_df['m'].tolist()

# 定数の定義
stock = {row.m:row.stock for row in stock_df.itertuples()}
gain = {row.p:row.gain for row in gain_df.itertuples()}
require = {(row.p,row.m):row.require for row in require_df.itertuples()}

# 数理最適化モデルの定義
problem = pulp.LpProblem('LP2', pulp.LpMaximize)

# 変数の定義
x = pulp.LpVariable.dicts('x', P, cat='Continuous')

# 制約式の定義
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum([require[p,m] * x[p] for p in P]) <= stock[m]

# 目的関数の定義    
problem += pulp.lpSum([gain[p] * x[p] for p in P])

# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

# 計算結果の表示
for p in P:
    print(p, x[p].value())

print('obj=', problem.objective.value())

Status: Optimal
p1 12.142857
p2 3.5714286
p3 7.4285714
p4 0.0
obj= 80.42857099999999


### **整数計画問題**

### コード全体

In [33]:
import pandas as pd
import pulp

# データの取得
require_df = pd.read_csv('requires.csv')
stock_df = pd.read_csv('stocks.csv')
gain_df = pd.read_csv('gains.csv')

# 集合の定義
P = gain_df['p'].tolist()
M = stock_df['m'].tolist()

# 定数の定義
stock = {row.m:row.stock for row in stock_df.itertuples()}
gain = {row.p:row.gain for row in gain_df.itertuples()}
require = {(row.p,row.m):row.require for row in require_df.itertuples()}

# 数理最適化モデルの定義
problem = pulp.LpProblem('IP', pulp.LpMaximize) # 変更点（任意）

# 変数の定義
x = pulp.LpVariable.dicts('x', P, cat='Integer') # 変更点

# 制約式の定義
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum([require[p,m] * x[p] for p in P]) <= stock[m]

# 目的関数の定義    
problem += pulp.lpSum([gain[p] * x[p] for p in P])

# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

# 計算結果の表示
for p in P:
    print(p, x[p].value())

print('obj=', problem.objective.value())

Status: Optimal
p1 13.0
p2 3.0
p3 7.0
p4 -0.0
obj= 79.0
